In [2]:
%load_ext autoreload
%autoreload 2
import os
import motti
motti.append_current_dir(os.path.abspath(''))

In [4]:
from dataset import ACDCDataset
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import h5py

In [5]:
train_set = ACDCDataset(root_dir="../../data/ACDC/database/training/")
train_set.configure_usage(
    use_es_gt=True,
    use_ed_gt=True
)

test_set = ACDCDataset(root_dir="../../data/ACDC/database/testing/")
test_set.configure_usage(
    use_es_gt=True,
    use_ed_gt=True
)


In [6]:
def mask2quadra(x, border=2) -> np.ndarray:
    w, h, z = x.shape
    non_black_pixels = np.where(x > 0)
    if len(non_black_pixels[0]) == 0:
        return None 
    
    top, bottom = min(non_black_pixels[0]), max(non_black_pixels[0])
    left, right = min(non_black_pixels[1]), max(non_black_pixels[1])
    
    width = right - left
    height = bottom - top
    
    side = max(width, height) + 2 * border
    anchor = (side-height) // 2, (side-width) // 2
    
    new_x = np.zeros(shape=(side, side, x.shape[-1])) 
    new_x[anchor[0]:anchor[0]+height, 
          anchor[1]:anchor[1]+width, 
          ...] = x[top:top+height, left:left+width, ...]

    quadra = np.empty((*new_x.shape, 4))
    for c in range(4):
        quadra[..., c] = np.where(new_x == c, 1, 0)
        
    return quadra

In [7]:
train_set[99].patient_info

{'ED': ' 1',
 'ES': ' 13',
 'Group': ' RV',
 'Height': ' 165',
 'NbFrame': ' 34',
 'Weight': ' 63'}

In [8]:
headers = ["ID", "Patient", "Phase", "Group", "NbFrame", "FrameIdx", "Height", "Weight", "H5path"]
df = pd.DataFrame(columns=headers)
d = {}
for i, h in enumerate(headers):
    d[h] =  None
    
n = 0
with h5py.File("acdc.h5", "a") as f:
    grp_name = "quadra_per_patient"
    if grp_name in list(f.keys()):
        grp = f[grp_name]
    else:
        grp = f.create_group(name=grp_name)
    
    for i in range(len(train_set)):
        patient = train_set[i]
        
        quadra_ed = mask2quadra(patient.ed_gt)
        d["ID"] = n
        d["Patient"] = patient.patient_dir.strip()
        d["Phase"] = "ed"
        d["Group"] = patient.patient_info["Group"].strip()
        d["FrameIdx"] = int(patient.patient_info["ED"])
        d["NbFrame"] = int(patient.patient_info["NbFrame"])
        d["Height"] = float(patient.patient_info["Height"])
        d["Weight"] = float(patient.patient_info["Weight"])
        d["H5path"] = f"{grp_name}/{n:03d}"
        df.loc[n] = d
        grp[f"{n:03d}"] = quadra_ed
        
        n += 1
        
        quadra_es = mask2quadra(patient.es_gt)
        d["ID"] = n
        d["Patient"] = patient.patient_dir.strip()
        d["Phase"] = "es"
        d["Group"] = patient.patient_info["Group"].strip()
        d["FrameIdx"] = int(patient.patient_info["ES"])
        d["NbFrame"] = int(patient.patient_info["NbFrame"])
        d["Height"] = float(patient.patient_info["Height"])
        d["Weight"] = float(patient.patient_info["Weight"])
        d["H5path"] = f"{grp_name}/{n:03d}"
        df.loc[n] = d
        grp[f"{n:03d}"] = quadra_es
        
        n += 1
        
    for i in range(len(test_set)):
        patient = test_set[i]
        
        quadra_ed = mask2quadra(patient.ed_gt)
        d["ID"] = n
        d["Patient"] = patient.patient_dir.strip()
        d["Phase"] = "ed"
        d["Group"] = patient.patient_info["Group"].strip()
        d["FrameIdx"] = int(patient.patient_info["ED"])
        d["NbFrame"] = int(patient.patient_info["NbFrame"])
        d["Height"] = float(patient.patient_info["Height"])
        d["Weight"] = float(patient.patient_info["Weight"])
        d["H5path"] = f"{grp_name}/{n:03d}"
        df.loc[n] = d
        grp[f"{n:03d}"] = quadra_ed
        
        n += 1
        
        quadra_es = mask2quadra(patient.es_gt)
        d["ID"] = n
        d["Patient"] = patient.patient_dir.strip()
        d["Phase"] = "es"
        d["Group"] = patient.patient_info["Group"].strip()
        d["FrameIdx"] = int(patient.patient_info["ES"])
        d["NbFrame"] = int(patient.patient_info["NbFrame"])
        d["Height"] = float(patient.patient_info["Height"])
        d["Weight"] = float(patient.patient_info["Weight"])
        d["H5path"] = f"{grp_name}/{n:03d}"
        df.loc[n] = d
        grp[f"{n:03d}"] = quadra_es
        
        n += 1

df.to_csv("quadra_per_patient.csv", index=False)

In [9]:
headers = ["ID", "Patient", "Phase", "Group", "NbFrame", "FrameIdx", "NbSlice", "SliceIdx", "Height", "Weight", "H5path"]
df = pd.DataFrame(columns=headers)
d = {}
for i, h in enumerate(headers):
    d[h] =  None
    
n = 0
with h5py.File("acdc.h5", "a") as f:
    grp_name = "quadra_per_slice"
    if grp_name in list(f.keys()):
        grp = f[grp_name]
    else:
        grp = f.create_group(name=grp_name)
    
    for i in range(len(train_set)):
        patient = train_set[i]
        
        quadra_ed = mask2quadra(patient.ed_gt)
        for j in range(quadra_ed.shape[2]):
            d["ID"] = n
            d["Patient"] = patient.patient_dir.strip()
            d["Phase"] = "ed"
            d["Group"] = patient.patient_info["Group"].strip()
            d["NbFrame"] = int(patient.patient_info["NbFrame"])
            d["FrameIdx"] = int(patient.patient_info["ED"])
            d["NbSlice"] = quadra_ed.shape[2]
            d["SliceIdx"] = j
            d["Height"] = float(patient.patient_info["Height"])
            d["Weight"] = float(patient.patient_info["Weight"])
            d["H5path"] = f"{grp_name}/{n:04d}"
            df.loc[n] = d
            grp[f"{n:04d}"] = quadra_ed[..., j, :]
            
            n += 1
            
        quadra_es = mask2quadra(patient.es_gt)
        for j in range(quadra_es.shape[2]):
            d["ID"] = n
            d["Patient"] = patient.patient_dir.strip()
            d["Phase"] = "es"
            d["Group"] = patient.patient_info["Group"].strip()
            d["NbFrame"] = int(patient.patient_info["NbFrame"])
            d["FrameIdx"] = int(patient.patient_info["ES"])
            d["NbSlice"] = quadra_es.shape[2]
            d["SliceIdx"] = j
            d["Height"] = float(patient.patient_info["Height"])
            d["Weight"] = float(patient.patient_info["Weight"])
            d["H5path"] = f"{grp_name}/{n:04d}"
            df.loc[n] = d
            grp[f"{n:04d}"] = quadra_es[..., j, :]
            
            n += 1

    for i in range(len(test_set)):
        patient = test_set[i]
        
        quadra_ed = mask2quadra(patient.ed_gt)
        for j in range(quadra_ed.shape[2]):
            d["ID"] = n
            d["Patient"] = patient.patient_dir.strip()
            d["Phase"] = "ed"
            d["Group"] = patient.patient_info["Group"].strip()
            d["NbFrame"] = int(patient.patient_info["NbFrame"])
            d["FrameIdx"] = int(patient.patient_info["ED"])
            d["NbSlice"] = quadra_ed.shape[2]
            d["SliceIdx"] = j
            d["Height"] = float(patient.patient_info["Height"])
            d["Weight"] = float(patient.patient_info["Weight"])
            d["H5path"] = f"{grp_name}/{n:04d}"
            df.loc[n] = d
            grp[f"{n:04d}"] = quadra_ed[..., j, :]
            
            n += 1
            
        quadra_es = mask2quadra(patient.es_gt)
        for j in range(quadra_es.shape[2]):
            d["ID"] = n
            d["Patient"] = patient.patient_dir.strip()
            d["Phase"] = "es"
            d["Group"] = patient.patient_info["Group"].strip()
            d["NbFrame"] = int(patient.patient_info["NbFrame"])
            d["FrameIdx"] = int(patient.patient_info["ES"])
            d["NbSlice"] = quadra_es.shape[2]
            d["SliceIdx"] = j
            d["Height"] = float(patient.patient_info["Height"])
            d["Weight"] = float(patient.patient_info["Weight"])
            d["H5path"] = f"{grp_name}/{n:04d}"
            df.loc[n] = d
            grp[f"{n:04d}"] = quadra_es[..., j, :]
            
            n += 1

df.to_csv("quadra_per_slice.csv", index=False)